In [ ]:
#Requirements

!pip install numpy
!pip install pandas
!pip install sklearn
!pip install seaborn
!pip install rfpimp
!pip install eli5
!pip install xgboost
!pip install matplotlib

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBClassifier
from xgboost import plot_importance
from matplotlib import pyplot as plt

np.random.seed(seed = 42)

In [ ]:
# Read CSV
task_file = '/kaggle/input/task_data.csv'
with open(task_file, mode = 'rb') as f:
    data = pd.read_csv(f)
    
# Extract names of each column (using pandas)
headers = np.array(list(data.columns.values))
names = headers[2:]
y = data.class_label
X = pd.DataFrame(data,columns=names)


In [ ]:
print(headers)

In [ ]:
print(names)

As per problem statement, the emphasis is on identifying the importance of a particular feature in the machine learning model. This is a classification problem wherein we need to identify the important feature and we need to justify the reasons by showing why we have chosen a specific feature and how it turns out to be important feature. Identifying the important features is very important due to the followig reasons:
    1. It will help us improve the model's by concentrating on important features (variables)
    2. It will help us get rid of features that are not relevant or which don not contribute the model's performance.
    
Feature importance can be implemented using various model's the popular among them being Random forest, XGBoost, Linear Regression. My idea here is that the better the model is, the more we can rely on the feature importance and how to interepret the problem. I will explain the reason for chosing the model as we proceed.

## Exploratory data analysis

In [ ]:
print(data.describe())

In [ ]:
sns.heatmap(X.assign(target = y).corr().round(2), cmap = 'Blues', annot = True).set_title('Correlation matrix', fontsize = 14)

# Models:

## a ) Random Forest

Since Decision trees can be used for better interpretability of the model, the downside is that it works well if the depth is very small. Since the current dataset has only 400 data points, it might be easy to interpret the model using Decision tree, but if the aspect of scalebility is considered it might not be a best approach and this is where Random Forest comes into picture. In Random forest the subset of data is chosen which is also called as bootstrap. These bootstrap are fed to decision trees with large depths randomly. The final result is obtained by counting the majority of the vote from all the decision trees. This technique is called as boot strapping or bagging. This also helps in overcoming bais.

In order to validate the Random forest model, I have used ```oob_score```. Out of bag score(oob_score) is used when the dataset is smaller in size. Since validation score requires us to keep apart certain part of da
ta, it is good to use oob_score since it works only on subset of decision trees and while the validation score is calculated on all the decision trees.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

rf =  RandomForestClassifier(n_estimators = 100,
                           n_jobs = -1,
                           oob_score = True,
                           bootstrap = True, random_state=42)

rf.fit(X, y)

In [ ]:
#print('Score :', rf.score(X_test,y_test))

print('Out of bag error:', rf.oob_score_)

As we can obersve that the model performed well on the training set. The ```oob_score``` obtained is also good. I will now check which feature is more important and try to visulaize it. This will help us understand which feature has to be given more importance.

In Scikit learn, we can use the feature importance by just using the decision tree which can help us in give some prior intution of th features. Decision Tree is one of the machine learning algorithms that is used for a range of classification and
regression problems and more specifically in the decision analysis problems. Decision tree provides a tree-like structure which comprises od decisions that can be made based on a hypothesis. A decision tree is represented in an upside-down where each node represents a feature also called attribute and each brach also called link to the nodes is represents a decision or condition for selecting the specific node. The top node of the tree is called the root node and the bottom-most nodes are called the leaf
nodes. The main idea behind using a decision tree is to represent the whole data in the form of a tree.  
    It has some advantages such as it is faster and it is easier to implement. The downside is it suffers from bias problem due to high cardinality and inflate the importance of continous featues.

In [ ]:
# function for creating a feature importance dataframe
def imp_df(column_names, importances):
    df = pd.DataFrame({'feature': column_names,
                       'feature_importance': importances}) \
           .sort_values('feature_importance', ascending = False) \
           .reset_index(drop = True)
    return df

# plotting a feature importance dataframe (horizontal barchart)
def var_imp_plot(imp_df, title):
    imp_df.columns = ['feature', 'feature_importance']
    sns.barplot(x = 'feature_importance', y = 'feature', data = imp_df, orient = 'h', color = 'green') \
       .set_title(title, fontsize = 20)

In [ ]:
base_imp = imp_df(X.columns, rf.feature_importances_)
base_imp

In [ ]:
var_imp_plot(base_imp, 'Default feature importance (scikit-learn)')

Using the default feature importance of Scikit-learn we can get the below mentioned graph. As we can notice that the top features are from ````Sensor8, Sensor4 and Sensor6```` .



[ Reference : https://scikit-learn.org/stable/modules/feature_selection.html ]


This default feature importance doesn't gives a proper picture of feature importance. This is because the there is bias involved in this method .[ Reference https://www.stat.berkeley.edu/~breiman/RandomForests/cc_home.htm#varimp ]. Another issue is the collinearity and the inflation towards the continous values.

Hence we need to find a better method.

# Other method to check the feature importance.

There are other ways to check the feature importance and I have chosen the following:


 ###   1. Permutation feature importance : 
        In this method we first train the model and calculate the score by passing validation set (or OOB set in case of Random Forest). We call this score as benchmark result. We then re shuffle the values from one feature in the dataset and then pass it to the model to calcualte the score again. We then calculate the feature importance score which the the difference between the score obtained after shuffling the data and benchmark result. We repeat this process for all the features. 
        This method has advantages like it is applicable to any model, reasonably efficient, reliable technique and no need to retrain the model at each modification of the dataset. The downside is it is more computationally expensive than default feature_importances permutation importance overestimates the importance of correlated predictors - Strobl et al (2008)
        
        
        [ Reference :https://explained.ai/rf-importance/index.html#5 ]

In [ ]:
from rfpimp import plot_corr_heatmap
viz = plot_corr_heatmap(X, figsize=(7,7))
viz.view()

In [ ]:
#Referring the code from https://explained.ai/rf-importance/index.html#5
from sklearn.metrics import r2_score
from rfpimp import permutation_importances

def r2(rf, X, y):
    return r2_score(y, rf.predict(X))

perm_imp_rfpimp = permutation_importances(rf, X, y, r2)
perm_imp_rfpimp.reset_index(drop = False, inplace = True)

In [ ]:
#Referring the code from https://explained.ai/rf-importance/index.html#5

from rfpimp import *
rf_pimp = RandomForestClassifier(n_estimators=100, n_jobs=-1)
%time I = oob_importances(rf_pimp, X, y)

In [ ]:
%time I = cv_importances(rf_pimp, X, y, k=5)

In [ ]:
%time I = dropcol_importances(rf_pimp, X, y)

In [ ]:
var_imp_plot(perm_imp_rfpimp, 'Permutation feature importance (rfpimp)')

As we can notice that the permutation method gives a different result when compared to the Scikit learn's default feature importance. In this result the ```sensor 6 ``` turned out to be more important. Even though ```sensor 8 ``` is still an important feature and is among the top 4 values, this helped us to overcome the issue of collinearity which the default feature importance method suffer's with

As we can notice that the time take to get the feature importance using permutation method is much less. This will be beneficial when the dataset is very small as in this case. However this method will suffer badly when compared to default feature importance method when the size of the data increases. Hence its not scalable.

Another feature importance method is using eli5 which is another option present in scikit learn.
[Reference : http://eli5.readthedocs.io/en/latest/blackbox/permutation_importance.html]

This method shows us a way where in we can check how the score will reduce if a particular feature is not present. Since the number of features present is not more for this problem, this method is not power intensive(resource consumption), otherwise it might suffer if the number of features increase. 

In [ ]:
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(rf, cv = None, refit = False, n_iter = 50).fit(X, y)
perm_imp_eli5 = imp_df(X.columns, perm.feature_importances_)
var_imp_plot(perm_imp_eli5, 'Permutation feature importance (eli5)')

In [ ]:
eli5.show_weights(perm)

As we can notice that by using this method we got the similar result as we obtained via rfpimp. This gives us a better undertanding that the ```sensor 6 ``` is an important feature. It has more weightage.

There are other feature selection methods mentioned in Scikit learn such as ```lasso```  but I haven't used them. 

[Reference : https://scikit-learn.org/stable/auto_examples/feature_selection/plot_select_from_model_boston.html#sphx-glr-auto-examples-feature-selection-plot-select-from-model-boston-py ] 

### 2.  Check the feature importance by dropping a column
I will now check how the model behaves if we drop certain coloumn features for training. This helps us justify our previous benchamarks and results that we have obtained. We will check if the ```sensor6``` feature turns out to be the important one which we have see above.

In [ ]:
from sklearn.base import clone 

def drop_col_feat_imp(model, X_train, y_train, random_state = 42):
    
    # clone the model to have the exact same specification as the one initially trained
    model_clone = clone(model)
    # set random_state for comparability
    model_clone.random_state = random_state
    # training and scoring the benchmark model
    model_clone.fit(X_train, y_train)
    benchmark_score = model_clone.score(X_train, y_train)
    # list for storing feature importances
    importances = []
    
    # iterating over all columns and storing feature importance (difference between benchmark and new model)
    for col in X_train.columns:
        model_clone = clone(model)
        model_clone.random_state = random_state
        model_clone.fit(X_train.drop(col, axis = 1), y_train)
        drop_col_score = model_clone.score(X_train.drop(col, axis = 1), y_train)
        importances.append(benchmark_score - drop_col_score)
    
    importances_df = imp_df(X_train.columns, importances)
    return importances_df

In [ ]:
from sklearn.ensemble import RandomForestRegressor
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.8, random_state = 42)

rf_1 =  RandomForestRegressor(n_estimators = 100,
                           n_jobs = -1,
                           oob_score = True,
                           bootstrap = True, random_state=42)

rf_1.fit(X_train, y_train)
%time drop_imp = drop_col_feat_imp(rf_1, X_train, y_train)
print(drop_imp)
var_imp_plot(drop_imp, 'Drop Column feature importance')

Now as we can see that certain features if dropped causes high impact both positively and negatively. As we can see sensors ```8,5,6``` and sensors ```4,0,9``` causes high impact on the overall model performance. This is one of the most important and accuracte feature selection method. The negative importance in this case means the model performance will improve if we remove these features. Another surprising thing to note is that ```sensor 4``` which was one of the important features in observed previously, turns out to be a negative impacting feature.

This method has one downfall which is comptutation cost. The reason for chosing random forest regressor in this method was that I wanted to see how dropping each feature causes the impact on the model. If we use the random forest classification, it will simply classify either 1 or -1 and we cannot derive any meaning out of it. 

## b) XGBoost classifier

In [ ]:
xgb = XGBClassifier(n_estimators = 100,
                           n_jobs = -1,
                           oob_score = True,
                           bootstrap = True, random_state=42)
xgb.fit(X, y)
# feature importance
print(xgb.feature_importances_)

# plot using default scikit learn feature selection 
plt.bar(range(len(xgb.feature_importances_)), xgb.feature_importances_)
plt.show()

In [ ]:
# plot feature importance using XGBOOST 
plot_importance(xgb)
plt.show()

# Conclusion : 
    From the above I can conclude that sensor 6 turns out to be an important feature. I have also explained the reasons for chosing the models and methods. The permutation method helps us to identify the feature importance by eliminating bias, on the other hand I explained the downside of it as well.
    
    Alternatively, I have used XGBoost feature importance and it turned out that the scikit learn default feature_importance gives the same result as previous ones but surprisingly, XGBOOST feature importance gives similar results as seen in the permuatation method because in boosting as we saw there is not much correlation between the features , boosting will use the feature that is not correlated since correlated feature will not be used in the splitting process.